In [ ]:
!gdown 1owVOM3V94bC8v2N8s7n56ciVzVbSzYtd -O hw1_data.zip
!unzip ./hw1_data.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: hw1_data/p2_data/mini/train/14643.jpg  
  inflating: hw1_data/p2_data/mini/train/16384.jpg  
  inflating: hw1_data/p2_data/mini/train/36721.jpg  
  inflating: hw1_data/p2_data/mini/train/13494.jpg  
  inflating: hw1_data/p2_data/mini/train/22689.jpg  
  inflating: hw1_data/p2_data/mini/train/25214.jpg  
  inflating: hw1_data/p2_data/mini/train/34194.jpg  
  inflating: hw1_data/p2_data/mini/train/30094.jpg  
  inflating: hw1_data/p2_data/mini/train/18423.jpg  
  inflating: hw1_data/p2_data/mini/train/12763.jpg  
  inflating: hw1_data/p2_data/mini/train/22852.jpg  
  inflating: hw1_data/p2_data/mini/train/15112.jpg  
  inflating: hw1_data/p2_data/mini/train/38279.jpg  
  inflating: hw1_data/p2_data/mini/train/15891.jpg  
  inflating: hw1_data/p2_data/mini/train/6020.jpg  
  inflating: hw1_data/p2_data/mini/train/8130.jpg  
  inflating: hw1_data/p2_data/mini/train/27684.jpg  
  inflating: hw1_data/p2_data/mini/train/1063.jpg  
  inflating: hw1_data/p2_dat

In [ ]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import DatasetFolder, VisionDataset
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random
import pandas as pd
import glob
import scipy.misc
import imageio
import argparse

In [ ]:
# import sys
# sys.path.append('/kaggle/input/byol-pytorch')
from byol_pytorch import BYOL

In [ ]:
np.random.seed(10901041)
torch.manual_seed(10901041)

import gc
gc.collect

<function gc.collect(generation=2)>

In [ ]:
class Dataset(Dataset):
    def __init__(self, path):
        super(Dataset).__init__()
        self.path = path
        self.files = sorted([p for p in os.listdir(path)])
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        name = self.files[i]
        image = self.transform(Image.open(os.path.join(self.path, name)))
        return image

In [ ]:
train_set = Dataset(path="./hw1_data/p2_data/mini/train")
train_loader = DataLoader(train_set, shuffle=True, pin_memory=True, batch_size=32)
print(len(train_set)) # Should print

38400


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()
resnet = models.resnet50(pretrained=False)
# resnet = resnet.to(device)

cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
learner = BYOL(
    resnet,
    image_size=128,
    hidden_layer="avgpool",
).to(device)
optimizer = torch.optim.Adam(learner.parameters(), lr=3e-4)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
best_loss = 5
num_epoch = 100
for epoch in range(num_epoch):
    # Train
    resnet.train()
    train_loss = []
    train_acc = []
    for images in tqdm(train_loader):
        loss = learner(images.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    t_loss = sum(train_loss) / len(train_loss)
    print(f"[ Train | {epoch + 1:03d}/{num_epoch:03d} ] loss = {t_loss:.5f}")

    if epoch % 10 == 9:
        torch.save(resnet.state_dict(), f"p2_BYOL_{epoch}.pt")
    if t_loss < best_loss:
        print(f"best model: epoch {epoch}, save model")
        epoch_state = {
            'epoch': epoch,
            'resnet_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
        }
        torch.save(epoch_state, f"p2_best_byol_state.pt")
        torch.save(resnet.state_dict(), f"p2_BYOL_pretrain.pt")
        best_loss = t_loss
    else:
        print(f"no improve: epoch {epoch}.")

  0%|          | 0/1200 [00:00<?, ?it/s]

[ Train | 001/100 ] loss = 1.43937
best model: epoch 0, save model


  0%|          | 0/1200 [00:00<?, ?it/s]

[ Train | 002/100 ] loss = 1.43119
best model: epoch 1, save model


  0%|          | 0/1200 [00:00<?, ?it/s]

...

[ Train | 098/100 ] loss = 1.39604
no improve: epoch 97.


  0%|          | 0/1200 [00:00<?, ?it/s]

[ Train | 099/100 ] loss = 1.39415
no improve: epoch 98.


  0%|          | 0/1200 [00:00<?, ?it/s]

[ Train | 100/100 ] loss = 1.39656
no improve: epoch 99.
